In [ ]:
import pandas as pd
import openai
# from googlesearch import search
from duckduckgo_search import DDGS
import requests
from bs4 import BeautifulSoup
import re

# Initialize OpenAI client

def get_naics_lookup(naics_path):
    """Load NAICS code lookup table"""
    return pd.read_excel(naics_path)[['2022 NAICS Code', '2022 NAICS Title']]

def web_search_company(query):
    """Search web for company information"""
    try:
        # Get top 3 Google results
        results = list(DDGS().text(
                keywords=query, 
                region="wt-wt", 
                safesearch=False, 
                max_results=10
        ))
        return results
    except Exception as e:
        print(f"Error searching {query}: {str(e)}")
    return None

from pydantic import BaseModel, Field, ValidationError
from openai import OpenAI

class NAICSClassification(BaseModel):
    code: str = Field(..., alias="NAICS Code")
    confidence: float = Field(..., ge=0.0, le=1.0, alias="Confidence Percentage")
    reason: str = Field(..., alias="Brief Reason")

def analyze_with_gpt(context: str, company_name: str) -> tuple:
    """Get validated NAICS classification using GPT-3.5-turbo"""
    client = OpenAI(api_key=api_key)
    
    prompt = f"""Analyze this company information and return NAICS classification in JSON format:
    Company: {company_name}
    Context: {context}
    
    Required JSON format:
    {{
        "NAICS Code": "string",
        "Confidence Percentage": 0.0-1.0,
        "Brief Reason": "string"
    }}"""
    
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You are an expert at analyzing NAICS codes. Return valid JSON."},
                {"role": "user", "content": prompt}
            ],
            response_format={"type": "json_object"}
        )
        
        result = NAICSClassification.parse_raw(response.choices[0].message.content)
        return result.code, result.confidence, result.reason
        
    except ValidationError as e:
        print(f"Validation error: {str(e)}")
        return None, 0.0, "Invalid response format"
    except Exception as e:
        print(f"API error: {str(e)}")
        return None, 0.0, "Analysis failed"

def keyword_match_naics(company_name, naics_df):
    """Fallback keyword matching algorithm"""
    keywords = re.findall(r'\b\w{4,}\b', company_name.lower())
    
    # Fill NaN values for safe string matching
    titles = naics_df['2022 NAICS Title'].fillna('').str.lower()
    
    # Find best matching NAICS description
    matches = titles.apply(
        lambda desc: sum(1 for word in keywords if word in desc)
    )
    
    if matches.max() > 0:
        best_match = naics_df.loc[matches.idxmax()]
        return best_match['2022 NAICS Code'], 0.5, "Keyword match"
    return None, 0.0, "No match found"


def process_company_directory(company_path, naics_df):
    """Main processing function"""
    df = pd.read_excel(company_path)
    
    for index, row in df.iterrows():
        company_name = row['Company']
        city = row['City']
        website = row['Web Site']
        web_content = web_search_company(f'{company_name} {city} {website}') 
        
        # Try web-based analysis first
        if web_content:
            code, confidence, reason = analyze_with_gpt(web_content, company_name)
            source = 'Web Analysis'
        else:  # Fallback to keyword matching
            code, confidence, reason = keyword_match_naics(company_name, naics_df)
            source = 'Keyword Match'
        
        # Validate NAICS code exists
        if code and code in naics_df['2022 NAICS Code'].values:
            description = naics_df[naics_df['2022 NAICS Code'] == code]['2022 NAICS Title'].values[0]
        else:
            code, description, confidence = None, None, 0.0
        
        # Update dataframe
        df.at[index, 'NAICS Code'] = code
        df.at[index, 'NAICS Description'] = description
        df.at[index, 'Confidence'] = confidence
        df.at[index, 'Source Method'] = f"{source}: {reason}"
    
    # Save results
    output_path = company_path.replace('.xlsx', '_processed.xlsx')
    df.to_excel(output_path, index=False)
    return output_path

if __name__ == "__main__":
    naics_path = '/Users/phytal/Documents/vdr/2022_NAICS_Structure.xlsx'
    company_path = '/Users/phytal/Documents/vdr/End Customer A-B Accounts by Industry Report_241122.xlsx'
    
    naics_df = get_naics_lookup(naics_path)
    print(naics_df.head())
    output_file = process_company_directory(company_path, naics_df)
    
    print(f"Processing complete. Results saved to: {output_file}")



  2022 NAICS Code                             2022 NAICS Title
0              11  Agriculture, Forestry, Fishing and HuntingT
1             111                            Crop ProductionT 
2            1111                   Oilseed and Grain FarmingT
3           11111                             Soybean FarmingT
4          111110                              Soybean Farming
Error searching 1st Detect Webster http://www.1stdetect.com: HTTP Error 429: Too Many Requests


TypeError: argument of type 'float' is not iterable